<a href="https://colab.research.google.com/github/Chiamakac/Scraping-Tables/blob/main/CARE_EXTRACTION_02_01_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount the goolge drive or upload the pdf files to extract

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install -q pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.3 MB/s eta 0:00:00


In [4]:
pip install -q fuzzywuzzy

In [5]:
pip install -q python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.8 MB/s eta 0:00:00


In [ ]:
import os
import shutil
from pdfminer.layout import LTTextContainer, LTChar
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.high_level import extract_text

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LAParams, PDFResourceManager, PDFPageAggregator, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
import os

def extract_pages(pdf_path):
    # PDFMiner setup for extracting pages
    resource_manager = PDFResourceManager()
    device = PDFPageAggregator(resource_manager, laparams=LAParams())
    interpreter = PDFPageInterpreter(resource_manager, device)

    with open(pdf_path, 'rb') as file:
        for page in PDFPage.get_pages(file, check_extractable=False):
            interpreter.process_page(page)
            layout = device.get_result()
            yield layout

def process_pdf_folder(input_folder, output_folder):
    # Iterate over files in the specified folder
    for root, _, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith('.pdf'):
                # Build paths
                input_file_path = os.path.join(root, filename)
                output_file_folder = os.path.join(output_folder, os.path.relpath(input_file_path, input_folder))
                os.makedirs(output_file_folder, exist_ok=True)

                for page_number, page_layout in enumerate(extract_pages(input_file_path), start=1):
                    output_file = os.path.join(output_file_folder, f'page{page_number}.txt')

                    with open(output_file, 'w') as file:
                        # Variables to track previous text and font size
                        previous_text = ''
                        previous_font_size = 0
                        merged_text = ''

                        for element in page_layout:
                            if isinstance(element, LTTextContainer):
                                for text_line in element:
                                    if isinstance(text_line, LTTextContainer):
                                        if len(text_line._objs) > 0:
                                            first_char = text_line._objs[0]
                                            if isinstance(first_char, LTChar):
                                                font_size = round(first_char.size)
                                                font_name = first_char.fontname
                                                font_color = first_char.graphicstate.ncolor
                                                text = element.get_text()

                                                # Check if the current text is different from the previous text
                                                if text != previous_text:
                                                    if font_size == previous_font_size:
                                                        # Merge the text with the previous one
                                                        merged_text += ' ' + text
                                                    else:
                                                        # Write the merged text as a single entry
                                                        if merged_text:
                                                            file.write(f"Font size: {previous_font_size}\n")
                                                            # file.write(f"Font color: {font_color}\n")
                                                            # file.write(f"Font name: {font_name}\n")
                                                            file.write(f"{merged_text}\n")
                                                            merged_text = ''

                                                        # Write the current text as a separate entry
                                                        file.write(f"Font size: {font_size}\n")
                                                        # file.write(f"Font color: {font_color}\n")
                                                        # file.write(f"Font name: {font_name}\n")
                                                        file.write(f"{text}\n")

                                                    # Update previous text and font size
                                                    previous_text = text
                                                    previous_font_size = font_size

                        # Write the final merged text, if any
                        if merged_text:
                            file.write(f"Font size: {previous_font_size}\n")
                            # file.write(f"Font color: {font_color}\n")
                            # file.write(f"Font name: {font_name}\n")
                            file.write(f"{merged_text}\n")

# Specify the path to the main folder containing subfolders and PDF files
main_folder = '/content/drive/foldername'
output_folder = 'output_folder_name'
os.makedirs(output_folder, exist_ok=True)

# Start processing the main folder and its subfolders
process_pdf_folder(main_folder, output_folder)


In [ ]:
#GETTING THE FONT SIZES HIGHER THAN THE FREQUENTLY USED

from pathlib import Path
from typing import Iterable, Any
from pdfminer.high_level import extract_pages
import os

def count_font_sizes(o: Any, size_dict: dict):
    # Recursively traverse the object to count font sizes
    if isinstance(o, Iterable):
        for i in o:
            count_font_sizes(i, size_dict)
    elif hasattr(o, 'size'):
        # Round the font size and update the dictionary
        size = round(o.size)
        if size in size_dict:
            size_dict[size] += 1
        else:
            size_dict[size] = 1

def store_font_sizes_greater_than_most_frequent(size_dict):
    # Find the most frequent font size
    most_frequent_size = max(size_dict, key=size_dict.get)
    # Filter sizes greater than the most frequent one
    filtered_sizes = {size: count for size, count in size_dict.items() if size > most_frequent_size}
    return filtered_sizes

def save_font_sizes_to_file(filtered_sizes, output_file):
    # Save font sizes to a text file
    with open(output_file, 'w') as file:
        for size, count in filtered_sizes.items():
            file.write(f"Font size: {size} \n")

def process_pdf_folder(input_folder, output_folder):
    # Iterate over PDF files in the input folder
    for pdf_path in Path(input_folder).rglob('*.pdf'):
        # Extract pages from the PDF
        pages = extract_pages(pdf_path)

        size_dict = {}  # Create an empty dictionary to store font size frequencies
        count_font_sizes(pages, size_dict)

        filtered_sizes = store_font_sizes_greater_than_most_frequent(size_dict)

        # Create output folder if it doesn't exist
        os.makedirs(output_folder, exist_ok=True)

        # Create output file path based on the PDF file name
        output_file = os.path.join(output_folder, f"{pdf_path.stem}_font_sizes.txt")

        # Save font sizes to the output file
        save_font_sizes_to_file(filtered_sizes, output_file)

# Specify the path to the input folder containing PDF files
input_folder = 'foldername'

# Specify the path to the output folder
output_folder = 'font_sizes_output'

# Start processing the input folder
process_pdf_folder(input_folder, output_folder)


In [ ]:
#comparing with the fontsizes above the frequently used and removing the  paragraphs
#with font sizes equal or lower than the frequently used. Then save the filterd file as output

import os
from fuzzywuzzy import fuzz

# Define the source folder path where your files are located.
source_folder_path = 'file.pdf'

# Define the destination folder path where you want to write the filtered files.
destination_folder_path = 'des_output'  # Create a new folder for filtered files

# Define the path to the file containing target font sizes.
target_phrases_file = 'target_font_sizes.txt'

# Define the similarity threshold (0 to 100) for fuzzy matching.
# Increase this value to make the comparison stricter.
similarity_threshold = 95

# Create the destination folder if it doesn't exist.
if not os.path.exists(destination_folder_path):
    os.makedirs(destination_folder_path)

# Function to check if a paragraph contains any of the target phrases using fuzzy matching.
def paragraph_contains_target_phrases(paragraph, target_phrases):
    for target_phrase in target_phrases:
        similarity_ratio = fuzz.partial_ratio(paragraph, target_phrase)
        if similarity_ratio >= similarity_threshold:
            return True
    return False

# Read target phrases from the file.
with open(target_phrases_file, 'r') as phrases_file:
    target_phrases = [line.strip() for line in phrases_file]

# Iterate through files in the source folder.
for root, _, files in os.walk(source_folder_path):
    for filename in files:
        source_file_path = os.path.join(root, filename)
        destination_file_path = os.path.join(destination_folder_path, os.path.relpath(source_file_path, source_folder_path))

        with open(source_file_path, 'r') as source_file, open(destination_file_path, 'w') as destination_file:
            paragraph = ""
            for line in source_file:
                # Check if the line is empty, denoting the end of a paragraph.
                if not line.strip():
                    # Check if the paragraph contains any of the target phrases.
                    if paragraph_contains_target_phrases(paragraph, target_phrases):
                        destination_file.write(paragraph + '\n')
                    paragraph = ""
                else:
                    paragraph += line
            # Check the last paragraph in the file.
            if paragraph_contains_target_phrases(paragraph, target_phrases):
                destination_file.write(paragraph + '\n')

In [ ]:
# # #write the first line(contains the font size) of any paragraph where the target phrases(Gold TOC) are found


import os
from fuzzywuzzy import fuzz

# Define the source folder path where your files are located.
source_folder_path = '/content/drive/MyDrive/CARE_MAIN/TopEng3/FILTERED_TopEng3'

# Define the destination folder path where you want to write the filtered files.
output_folder_path = 'output_filename'  # Create a new folder for filtered files

# Define the path to the file containing target phrases.
target_phrases_file = 'gold_section_header_file'

# Define the similarity threshold (0 to 100) for fuzzy matching.
# Increase this value to make the comparison stricter.
similarity_threshold = 95

# Create the destination folder if it doesn't exist.
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Function to check if a paragraph contains any of the target phrases using fuzzy matching.
def paragraph_contains_target_phrases(paragraph, target_phrases):
    for target_phrase in target_phrases:
        similarity_ratio = fuzz.partial_ratio(paragraph, target_phrase)
        if similarity_ratio >= similarity_threshold:
            return True
    return False

# Read target phrases from the file.
with open(target_phrases_file, 'r') as phrases_file:
    target_phrases = [line.strip() for line in phrases_file]

# Iterate through files in the source folder.
for root, _, files in os.walk(source_folder_path):
    for filename in files:
        source_file_path = os.path.join(root, filename)
        relative_path = os.path.relpath(source_file_path, source_folder_path)
        destination_file_path = os.path.join(output_folder_path, relative_path)

        # Create subdirectories in the output folder if they don't exist
        os.makedirs(os.path.dirname(destination_file_path), exist_ok=True)

        with open(source_file_path, 'r') as source_file, open(destination_file_path, 'w') as destination_file:
            paragraph = []
            for line in source_file:
                # Check if the line is empty, denoting the end of a paragraph.
                if not line.strip():
                    # Check if the paragraph contains any of the target phrases.
                    if paragraph_contains_target_phrases(' '.join(paragraph), target_phrases):
                        # Write the first line of the paragraph.
                        destination_file.write(paragraph[0] + '\n')
                    paragraph = []
                else:
                    paragraph.append(line)

In [ ]:
#Merging of the pages/files 'first line(contains the font size) of any paragraph where the target phrases(Gold section header) were found


import os

# Folder containing text files in subfolders to sort and merge
main_folder_path = 'filename'

# Output folder where the merged content of each subfolder will be saved
output_folder = 'merged_output_file'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through subfolders in the main folder
for subfolder_name in os.listdir(main_folder_path):
    subfolder_path = os.path.join(main_folder_path, subfolder_name)

    # Check if it's a subfolder
    if os.path.isdir(subfolder_path):
        # Output file where the merged content will be saved for each subfolder
        output_file = os.path.join(output_folder, f'{subfolder_name}_merged.txt')

        # Create a set to store unique sentences
        unique_sentences = set()

        # Get a list of text files in the subfolder
        text_files = [f for f in os.listdir(subfolder_path) if f.endswith('.txt')]

        # Sort the list of file names
        sorted_files = sorted(text_files)

        # Iterate through the sorted files and extract unique sentences
        for filename in sorted_files:
            file_path = os.path.join(subfolder_path, filename)

            # Open and read each text file
            with open(file_path, 'r') as input_file:
                content = input_file.read()

                # Split the content into sentences (assuming sentences are separated by periods)
                sentences = content.split('. ')

                # Add unique sentences to the set
                unique_sentences.update(sentences)

        # Open the output file for writing
        with open(output_file, 'w') as merged_file:
            # Write the unique sentences to the merged file
            for sentence in unique_sentences:
                merged_file.write(sentence)


In [ ]:
# Compare the output from the previous cell with the filtered file. Check for paragraphs that contains any of the target phrases(merged_section header) and start writing from the second line


# import os
from fuzzywuzzy import fuzz

# Define the source folder path where your files are located.
source_folder_path = 'FILTERED_file'

# Define the destination folder path where you want to write the filtered files.
destination_folder_path = 'ouput_file'  # Create a new folder for filtered files

# Define the path to the file containing target phrases.
target_phrases_file = 'merged_fontsizes file.txt'

# Define the similarity threshold (0 to 100) for fuzzy matching.
# Increase this value to make the comparison stricter.
similarity_threshold = 95

# Create the destination folder if it doesn't exist.
if not os.path.exists(destination_folder_path):
    os.makedirs(destination_folder_path)

# Function to check if a paragraph contains any of the target phrases using fuzzy matching.
def paragraph_contains_target_phrases(paragraph, target_phrases):
    for target_phrase in target_phrases:
        similarity_ratio = fuzz.partial_ratio(paragraph, target_phrase)
        if similarity_ratio >= similarity_threshold:
            return True
    return False

# Read target phrases from the file.
with open(target_phrases_file, 'r') as phrases_file:
    target_phrases = [line.strip() for line in phrases_file]

# Iterate through files in the source folder.
for root, _, files in os.walk(source_folder_path):
    for filename in files:
        source_file_path = os.path.join(root, filename)
        destination_file_path = os.path.join(destination_folder_path, os.path.relpath(source_file_path, source_folder_path))

        with open(source_file_path, 'r') as source_file, open(destination_file_path, 'w') as destination_file:
            paragraph = ""
            write_paragraph = False  # Flag to control writing from the second line onwards
            for line in source_file:
                # Check if the line is empty, denoting the end of a paragraph.
                if not line.strip():
                    # Check if the paragraph contains any of the target phrases.
                    if paragraph_contains_target_phrases(paragraph, target_phrases):
                        # Write the paragraph starting from the second line if the flag is set
                        lines = paragraph.split('\n')
                        if write_paragraph:
                            destination_file.write('\n'.join(lines[1:]) + '\n')
                        write_paragraph = False

                    paragraph = ""
                else:
                    paragraph += line
                    if paragraph_contains_target_phrases(paragraph, target_phrases):
                        write_paragraph = True

            # Check the last paragraph in the file.
            if paragraph_contains_target_phrases(paragraph, target_phrases):
                # Write the last paragraph starting from the second line if the flag is set
                lines = paragraph.split('\n')
                if write_paragraph:
                    destination_file.write('\n'.join(lines[1:]) + '\n')


In [ ]:
# # Write the paragraph to the output file if it has less than 9 words

import os

def process_folder(input_folder, output_folder):
    for item in os.listdir(input_folder):
        item_path = os.path.join(input_folder, item)

        if os.path.isdir(item_path):
            # If it's a subfolder, recursively process it
            subfolder_name = os.path.basename(item_path)
            subfolder_output = os.path.join(output_folder, subfolder_name)
            os.makedirs(subfolder_output, exist_ok=True)
            process_folder(item_path, subfolder_output)
        elif item.endswith('.txt'):
            # If it's a file, process it and save the output
            input_file_path = item_path
            output_file_path = os.path.join(output_folder, item)

            with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
                inside_paragraph = False
                paragraph_lines = []

                for line in input_file:
                    # Detect the end of a paragraph (an empty line)
                    if not line.strip():
                        # Check if the paragraph has less than 9 words
                        if len(paragraph_lines) < 9:
                            output_file.write(join_paragraph(paragraph_lines) + '\n')
                        inside_paragraph = False
                        paragraph_lines = []
                    else:
                        inside_paragraph = True
                        paragraph_lines.append(line.strip())

                # Ensure the last paragraph is processed
                if inside_paragraph and len(paragraph_lines) < 9:
                    output_file.write(join_paragraph(paragraph_lines) + '\n')

def join_paragraph(paragraph_lines):
    return ' '.join(paragraph_lines)

# Specify the path to the main folder containing subfolders and text files
main_folder = 'filepath'  # Replace with the actual main folder path

# Specify the path to the output folder
output_folder = 'FINAL'  # Replace with the desired output folder path

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Start processing the main folder and its subfolders
process_folder(main_folder, output_folder)


In [ ]:
import os
import pandas as pd

# Specify the path to the main folder containing subfolders and text files
main_folder = 'FINAL_file'  # Replace with the actual main folder path

# Create an empty list to store DataFrames
dfs = []

# Initialize variables to track the last folder
last_folder_name = None
last_df = None

# Define a function to extract the page number from a filename
def extract_page_number(filename):
    page_prefix = "page"
    if page_prefix in filename:
        start = filename.index(page_prefix) + len(page_prefix)
        end = filename.index('.', start)
        return int(filename[start:end])
    return 0  # Return 0 for filenames without a page number

# Iterate through subfolders and files
for folder, subfolders, files in os.walk(main_folder):
    for filename in files:
        if filename.endswith('.txt'):
            folder_name = os.path.basename(folder)

            # Sort the files in each folder based on page numbers
            file_list = sorted(files, key=extract_page_number)

            # Create an empty list to store DataFrames for this folder
            folder_dfs = []

            for filename in file_list:
                file_path = os.path.join(folder, filename)
                with open(file_path, 'r') as file:
                    content = file.readlines()

                    # Remove trailing empty lines and strip each line
                    cleaned_content = [line.strip() for line in content if line.strip()]

                    # Extract the page number (left side of the filename before the period)
                    page_number = extract_page_number(filename)

                    # Create a DataFrame for each file with the page number as an integer
                    df = pd.DataFrame({'Reports': [folder_name] * len(cleaned_content), 'Page Numbers': [page_number] * len(cleaned_content), 'Content': cleaned_content})
                    folder_dfs.append(df)

            # Concatenate DataFrames for this folder
            if folder_dfs:
                folder_df = pd.concat(folder_dfs, ignore_index=True)

                # Check if this folder is different from the last processed folder
                if folder_name != last_folder_name:
                    dfs.append(folder_df)
                    last_folder_name = folder_name
                else:
                    # Compare with the last DataFrame to prevent repetition
                    if last_df is None or not last_df.equals(folder_df):
                        dfs.append(folder_df)

                last_df = folder_df

# Concatenate all DataFrames into one
result_df = pd.concat(dfs, ignore_index=True)

# Convert 'Page Numbers' column to integers
result_df['Page Numbers'] = result_df['Page Numbers'].astype(int)

# Print the DataFrame
print(result_df)


In [ ]:
# convert to excel
with pd.ExcelWriter('file.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Files Details')

## **NOTE Extraction**

In [ ]:
import os

def process_folder(input_folder, output_folder):
    for item in os.listdir(input_folder):
        item_path = os.path.join(input_folder, item)

        if os.path.isdir(item_path):
            # If it's a subfolder, recursively process it
            subfolder_name = os.path.basename(item_path)
            subfolder_output = os.path.join(output_folder, subfolder_name)
            os.makedirs(subfolder_output, exist_ok=True)
            process_folder(item_path, subfolder_output)
        elif item.endswith('.txt'):
            # If it's a file, process it and save the output
            input_file_path = item_path

            # Define the output file path in the output folder
            output_file_path = os.path.join(output_folder, item)

            # Call the functions to process the file and save the output
            check_lines(input_file_path, output_file_path)
            check_lines_and_words(input_file_path, output_file_path)

def check_lines(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file:
        lines = input_file.readlines()
        total_lines = len(lines)
        i = 0
        current_line = ""

        with open(output_file_path, 'w') as output_file:
            while i < total_lines:
                words = lines[i].strip().split()

                if len(words) >= 2 and words[0].lower() == 'note' and words[1][0].isdigit():
                    current_line = lines[i].strip()

                    # Check if there is a third word
                    if len(words) >= 3:
                        output_file.write(current_line + '\n')
                    else:
                        # Join the current line with the next line since there's no third word
                        current_line += " " + lines[i + 2].strip()
                        output_file.write(current_line + '\n')

                    i += 1  # Move to the next line

                i += 1

def check_lines_and_words(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file:
        for line_number, line in enumerate(input_file, start=1):
            # Split the line into words
            words = line.strip().split()

            if len(words) >= 2 and len(words) < 10:  # Check if there are at least 2 words and less than 10 words
                # Check if the first word starts with a digit and ends with a period
                # or ends with a closing parenthesis ')' and the second word starts with an uppercase letter
                first_word = words[0]
                second_word = words[1]

                if (first_word[0].isdigit() and (first_word.endswith('.') or first_word.endswith(')')) and
                        second_word[0].isupper()):
                    with open(output_file_path, 'a') as output_file:
                        output_file.write(f"{line.strip()}\n")

# Specify the path to the main folder containing subfolders and text files
main_folder = 'extracted files'  # Replace with the actual main folder path

# Specify the path to the output folder
output_folder = 'output_filenmae'  # Replace with the desired output folder path

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Start processing the main folder and its subfolders
process_folder(main_folder, output_folder)


In [ ]:
# Outputing the Notes in a Dataframe
import os
import pandas as pd

# Specify the path to the main folder containing subfolders and text files
main_folder = 'filepath'  # Replace with the actual main folder path

# Create an empty list to store DataFrames
dfs = []

# Initialize variables to track the last folder
last_folder_name = None
last_df = None

# Define a function to extract the page number from a filename
def extract_page_number(filename):
    page_prefix = "page"
    if page_prefix in filename:
        start = filename.index(page_prefix) + len(page_prefix)
        end = filename.index('.', start)
        return int(filename[start:end])
    return 0  # Return 0 for filenames without a page number

# Iterate through subfolders and files
for folder, subfolders, files in os.walk(main_folder):
    for filename in files:
        if filename.endswith('.txt'):
            folder_name = os.path.basename(folder)

            # Sort the files in each folder based on page numbers
            file_list = sorted(files, key=extract_page_number)

            # Create an empty list to store DataFrames for this folder
            folder_dfs = []

            for filename in file_list:
                file_path = os.path.join(folder, filename)
                with open(file_path, 'r') as file:
                    content = file.readlines()

                    # Remove trailing empty lines and strip each line
                    cleaned_content = [line.strip() for line in content if line.strip()]

                    # Extract the page number (left side of the filename before the period)
                    page_number = extract_page_number(filename)

                    # Create a DataFrame for each file with the page number as an integer
                    df = pd.DataFrame({'Reports': [folder_name] * len(cleaned_content), 'Page Numbers': [page_number] * len(cleaned_content), 'Content': cleaned_content})
                    folder_dfs.append(df)

            # Concatenate DataFrames for this folder
            if folder_dfs:
                folder_df = pd.concat(folder_dfs, ignore_index=True)

                # Check if this folder is different from the last processed folder
                if folder_name != last_folder_name:
                    dfs.append(folder_df)
                    last_folder_name = folder_name
                else:
                    # Compare with the last DataFrame to prevent repetition
                    if last_df is None or not last_df.equals(folder_df):
                        dfs.append(folder_df)

                last_df = folder_df

# Concatenate all DataFrames into one
result_df = pd.concat(dfs, ignore_index=True)

# Convert 'Page Numbers' column to integers
result_df['Page Numbers'] = result_df['Page Numbers'].astype(int)

# Print the DataFrame
print(result_df)


In [ ]:
# convert to excel
with pd.ExcelWriter('note.xlsx') as writer:
    result_df.to_excel(writer, sheet_name='Files Details')

# **Table, Image extraction**

In [ ]:
pip install -qqq PyMuPDF

In [ ]:
pip install -qqq pillow

In [ ]:
pip install -q tabula

In [ ]:
pip install tabula-py pandas

In [ ]:
# import libraries
import fitz
import io
from PIL import Image, ImageDraw

In [ ]:
import os
import fitz  # PyMuPDF
from PIL import Image, ImageDraw
import io
from tabula import read_pdf

def extract_tables_and_images(pdf_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    # Open the PDF file
    with fitz.open(pdf_path) as pdf_doc:
        for page_num in range(pdf_doc.page_count):
            # Extract tables using tabula
            tables = read_pdf(pdf_path, pages=page_num + 1)

            # Save each table as an Excel file with page number
            for table_num, table in enumerate(tables, start=1):
                if table is not None and not table.empty:
                    table.to_excel(os.path.join(output_folder, f"table_tabula_page_{page_num + 1}_table_{table_num}.xlsx"), index=False)

            # Extract images
            page = pdf_doc[page_num]
            image_list = page.get_images()


            # Display and save images
            for image_index, img in enumerate(image_list, start=1):
                xref = img[0]
                image_bytes = pdf_doc.extract_image(xref)["image"]
                pil_image = Image.open(io.BytesIO(image_bytes))

                # Display image
                pil_image.show()

                # Save image
                image_path = os.path.join(output_folder, f"image_page_{page_num + 1}_image_{image_index}.png")
                pil_image.save(image_path)

                print(f"[+] Image {image_index} on page {page_num + 1} saved as {image_path}")

# Example usage
pdf_path = 'filepath.pdf'
output_folder = 'OUTPUT_FOLDER'
extract_tables_and_images(pdf_path, output_folder)